In [1]:
from pgmpy.base import DAG
import rpy2.robjects as robjects
import pandas as pd
import numpy as np
from rpy2.robjects import r, pandas2ri 
pandas2ri.activate()
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import plotly
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True) 
import plotly.graph_objs as go
from sklearn.metrics import mutual_info_score

In [2]:
robjects.r("""
    data_tofactor<-function(data){

        datacols = names(data)
       for (i in 1:ncol(data)) {
          data[,datacols[i]] <- factor(data[,datacols[i]])
        }
    return(data)
    }
""")

#R结构
robjects.r("""
        
        r_sturct_study<-function(self_r_data,bl,hos_id,method){
        
            library(bnlearn)
            
            self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
            train_size = ceiling(length(self_r_data[,1])*0.7)
            
            train_data=self_r_data[1:train_size,]
            

            dag = hc(train_data,score = method,blacklist = bl)
            
            arcs = arc.strength(dag,self_r_data)

            return(arcs)
        }
""")

#R结构
robjects.r("""
        
        r_sturct_study_wl<-function(self_r_data,bl,wl,hos_id,method){
        
            library(bnlearn)
            
            self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
            train_size = ceiling(length(self_r_data[,1])*0.7)
            
            train_data=self_r_data[1:train_size,]
            

            dag = hc(train_data,score = method,whitelist=wl,blacklist = bl)
            
            arcs = arc.strength(dag,self_r_data)
            
            return(arcs)
        }
""")

#定义个一个用来预测的R语言的函数
robjects.r("""
    predict_label<-function(r_dag,self_r_data,hos_id){
        library(bnlearn)
        
        model = model2network(r_dag)
        
        self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
        train_size = ceiling(length(self_r_data[,1])*0.7)
            
        
        
        train_data=self_r_data[1:train_size,nodes(model)]
        
        test_data = self_r_data[(train_size+1):nrow(self_r_data),nodes(model)]
    
        training_model = bn.fit(model,train_data)
        
        predicted = predict(training_model, node = "label", data = test_data)
        
        y_true = test_data[,"label"]
        
        out <- c(predicted ,y_true)
        
        return(out)
    }
""")
robjects.r("""
    dag_score<-function(dag,self_r_data,score,hos_id){
    
    model = model2network(dag)
    
    self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
    train_size = ceiling(length(self_r_data[,1])*0.7)
            
    train_data=self_r_data[1:train_size,nodes(model)]
   
    scr = score(model, train_data, type =score)
    
    return(scr)
    }

""")

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000001C951B94280> [RTYPES.CLOSXP]
R classes: ('function',)

In [3]:
def py_to_rdag(dag):
    #将python的DAG()转化为R语言
    rdag_str = ""
    for node in dag.nodes():
        tmp_str = "["+node
        if dag.get_parents(node)!=[]:
            tmp_str = tmp_str+"|"
            index = len
            for i in range(len(dag.get_parents(node))):
                if i == len(dag.get_parents(node))-1:
                    tmp_str+=dag.get_parents(node)[i]
                else:
                    tmp_str+=dag.get_parents(node)[i]+":"


        tmp_str+="]"
        rdag_str += tmp_str
    return rdag_str
def martix_list(edges_strength):
    #将矩阵图信息转换为from-to-val形式
    edges_list = []
    for from_node in feature:
        for to_node in feature:
            val = edges_strength.loc[from_node,to_node]
            if val!=0:
                edges_list.append([from_node,to_node,val])
    edges_list = pd.DataFrame(edges_list,columns=["from","to","val"]).sort_values(by="val")
    return edges_list
def list_to_martix(edges_list):
    #将from-to-val形式转换为矩阵图形式
    edges_strength = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int8)
    for i in range(len(dag_list)):
        from_node = edges_list["from"][i]
        to_node = edges_list["to"][i]
        val = edges_list["strength"][i]
        edges_strength.loc[from_node,to_node]=val
    edges_matrix = edges_strength.round(2)
    return edges_martix
def net_score(ed,data,hos_id,scr):
    ##转换为R字符串
    r_dag = py_to_rdag(ed)
    ##预测
    r_y = robjects.r['predict_label'](r_dag,data,hos_id)-1
    for i in range(len(r_y)):
        if r_y[i]<0:
            r_y[i]=0
        elif r_y[i]>1:
            r_y[i]=1

    split_size = int(len(r_y)*0.5) 
    
    y_pred = r_y[:split_size]
    
    y_true = r_y[split_size:]
    
    acc = accuracy_score(y_true,y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    
    dag_scr = robjects.r["dag_score"](r_dag,r_DB,scr,hos_id)[0]
    
    return acc,recall,auc,f1,dag_scr
def del_edges():
    pass
def to_info_martix(hos_id):
    data = DB[DB["hospitalid"]==hos_id]
    info_martix  =  pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    for col in data.columns:
        for index in data.columns:
            if col==index:
                info_martix.loc[index,col]=0
            else:
                info = mutual_info_score(data.loc[:,index],data.loc[:,col])
                info_martix.loc[index,col]=info
    return info_martix 


In [4]:
class FD_client():
    def __init__(self,cid,expert_bl):
        self.cid = cid
        self.score = {
            "acc":[],
            "recall":[],
            "auc":[],
            "f1":[],
            "score":[]}
        ##记录边的强度
        self.edges_strength = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
        ##记录边的频数
        self.edges_num = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
        
        self.info_list = martix_list(to_info_martix(self.cid).round(4))
        #构建互信息黑名单
        info_bl = []
        for i in range(len(self.info_list)):
            if self.info_list["val"][i]<0.001:
                info_bl.append([self.info_list["from"][i],self.info_list["to"][i]])
                
        bl = info_bl+expert_bl
        bl = pd.DataFrame(bl,columns=["from","to"])
        
        bl = robjects.conversion.py2rpy(bl)
        self.bl = bl
    def init_dag(self,method):
        ##第一轮初始化训练
        
        edges  = robjects.r['r_sturct_study'](r_DB,self.bl,self.cid,method)
        
        edges_list  = robjects.pandas2ri.rpy2py_dataframe(edges).sort_values(by="strength",ascending=False)
        
        dag = DAG()
        #强度列表 => DAG()
        self.edges_strength.iloc[:,:]=0
        self.edges_num.iloc[:,:]=0
        for i in range(len(edges_list )):
            #强度列表转为DAG()
            dag.add_edge(edges_list ["from"][i],edges_list ["to"][i])
            #强度记录
            self.edges_strength.loc[[edges_list ["from"][i]],[edges_list ["to"][i]]]=edges_list ["strength"][i]
            #频数记录
            self.edges_num.loc[[edges_list ["from"][i]],[edges_list ["to"][i]]]=1

        self.dag = dag
        #进行性能计算
        
        dag_score= net_score(self.dag,r_DB,self.cid,method)
        
        self.score["acc"].append(dag_score[0])
        self.score["recall"].append(dag_score[1])
        self.score["auc"].append(dag_score[2])
        self.score["f1"].append(dag_score[3])
        self.score["score"].append(dag_score[4])
        
        return 
    def update(self,wl,method):
        ##进行更新,基于网络评分的回退筛选机制
        ##基于wl学习的网络生成DAG图进行判断
        dag = DAG()
        
        edges  = robjects.r['r_sturct_study_wl'](r_DB,self.bl,wl,self.cid,method)
        
        edges_list = robjects.pandas2ri.rpy2py_dataframe(edges).sort_values(by="strength",ascending=False)
        
        #将强度列表转换为DAG()
        for i in range(len(edges_list)):
            dag.add_edge(edges_list["from"][i],edges_list["to"][i])
        
        dag_score= net_score(dag,r_DB,self.cid,method)
        
        """判断是否加入白名单"""
        if 1==1:
            self.edges_strength.iloc[:,:]=0
            self.edges_num.iloc[:,:]=0
            #更新边强度什么的
            self.dag = dag
            for i in range(len(edges_list)):
                self.edges_strength.loc[[edges_list["from"][i]],[edges_list ["to"][i]]]=edges_list ["strength"][i]
                self.edges_num.loc[[edges_list["from"][i]],[edges_list ["to"][i]]]=1

                dag.add_edge(edges_list ["from"][i],edges_list ["to"][i])
            #将只与label有关的节点加入数据
            
            for i in range(len(edges_list)):
                
                self.edges_strength.loc[[edges_list["from"][i]],[edges_list ["to"][i]]]=edges_list ["strength"][i]
                self.edges_num.loc[[edges_list["from"][i]],[edges_list ["to"][i]]]=1
                dag.add_edge(edges_list ["from"][i],edges_list ["to"][i])
        else:
            #则将重复记录旧分数，因为网络没有变所以不更改
            dag_score = [
                self.score["acc"][-1],
                self.score["recall"][-1],
                self.score["auc"][-1],
                self.score["f1"][-1],
                self.score["score"][-1],
            ]
        
        self.score["acc"].append(dag_score[0])
        self.score["recall"].append(dag_score[1])
        self.score["auc"].append(dag_score[2])
        self.score["f1"].append(dag_score[3])
        self.score["score"].append(dag_score[4])
        return

In [5]:
class FD_sever():
    def __init__(self,method,learning_rate):
        self.method = method
        self.learning_rate = learning_rate
        self.round_num=+1
        self.score = {
            "acc":[],
            "recall":[],
            "auc":[],
            "f1":[],
            "score":[]}
        
        
    ##在这里定义融合策略
    def num_method(self,edges_num):
        #转换为二维的dataframe格式
        edges_list = martix_list(edges_num)
        wl = []
        print(edges_list)
        for i in range(len(edges_list)):
            if edges_list["val"][i] >=7:
                n1=edges_list.iloc[i,0]
                n2=edges_list.iloc[i,1]
                wl.append((n1,n2))
        return wl
    def select_wl(self,edges_strength,edges_num,weight,mehtod):
        learing_rate = self.round_num*self.learning_rate
        self.round_num+=1
        """
        method:
            0:频数
            1:强度
            2:基于分数和网络频数的剪枝策略
            3:强度+频数
            4:与label有关的强度集筛选
        learing_rare:
            学习步长-每次传递多少条边
        weught:
            数据量的权重
        return 一个转换
        """
        fuse_DAG = DAG()
        
        for col in feature:
            for idx in feature:
                #根据频数判断，如果有相同方向的则取频数较大的边
                if edges_num[col][idx]!=0 and edges_num[col][idx]<=edges_num[idx][col]:
                    print(col,idx)
                    edges_num.loc[idx,col] = 0
                    edges_strength.loc[idx,col]=0
                    
        if mehtod==0:
            #选取一定长度的
            wl = self.num_method(edges_num)
        elif method==1:
            pass
        elif method==2:
            pass
        elif method==3:
            pass
        elif methid==4:
            pass
        else:
            print("融合策略码错误")
        #f返回结果
        
        fuse_DAG.add_edges_from(wl)
        wl=pd.DataFrame(wl,columns=["from","to"])
        return fuse_DAG,wl

In [6]:
#定义计算熵的函数
def ent(data):
    prob1 = pd.value_counts(data) / len(data)
    return sum(np.log2(prob1) * prob1 * (-1))
def gain(data,str1,str2):
    e1 = data.groupby(str1).apply(lambda x:ent(x[str2]))
    p1 = pd.value_counts(data[str1]) / len(data[str1])
    e2 = sum(e1 * p1)
    return ent(data[str2]) - e2

In [7]:
DB= pd.read_csv("c:data/site_16topaucoutlinePmmImpData_dis.csv").drop(columns="Unnamed: 0",axis = 1)
feature_list = DB.drop(columns="hospitalid",axis=1)
DB=DB.sample(frac=1,random_state=11).reset_index(drop=True)
r_DB=robjects.pandas2ri.py2rpy(DB)
r_DB =robjects.r['data_tofactor'](r_DB)

hos_id = [420,142,122,435,390,227,144,140,396,141]
hos_id = np.sort(hos_id)
# hos_id = list(set(DB_1.iloc[:,0]))
hos_id=np.sort(hos_id)
feature = DB.drop(columns="hospitalid",axis=1).columns
  #设置黑名单
bl =[]
for i in feature_list:
    bl.append(["label",i])
    bl.append([i,"sex"])
    bl.append([i,"race"])
    bl.append([i,"bmi"])



#初始化客户端
scr = "k2"
client_num = len(hos_id)
clients = []
for i in range(client_num):
    clients.append(FD_client(hos_id[i],bl))
    clients[i].init_dag(scr)
    
#初始化中心
learning_rate = 2


In [8]:
#学习步长和学习轮次
fuse_dag_list = []
sever = FD_sever(scr,learning_rate)
random_num =2
for i in range(random_num):
    #初始化融合强度矩阵，融合边强度，白名单列表
    fuse_edges_num = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    fuse_edges_strength = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    wl =[]
    fuse_dag = DAG()
    #汇集
    for k in range(client_num):
        fuse_edges_num+=clients[k].edges_num
        fuse_edges_strength+=clients[k].edges_strength
    
    fuse_dag,wl=sever.select_wl(fuse_edges_strength,fuse_edges_num,1,0)
   #看融合不去除边的网络在数据集上的性能
    fuse_dag_list.append(fuse_dag)
    #转换白名单
    wl_r =robjects.conversion.py2rpy(wl)
    #更新
    for k in range(client_num):
        clients[k].update(wl_r,scr)

respiration heartrate
respiration LAB5
LAB2 LAB42
LAB3 LAB38
LAB3 COM11
LAB4 LAB18
LAB4 LAB25
LAB4 PRO3
LAB7 LAB11
LAB7 LAB16
LAB8 LAB19
LAB8 PRO8
LAB10 LAB12
LAB10 LAB18
LAB10 LAB39
LAB10 LAB59
LAB12 LAB15
LAB12 LAB56
LAB12 PRO8
LAB14 LAB4
LAB14 LAB20
LAB15 LAB4
LAB16 LAB2
LAB18 LAB50
LAB19 age
LAB19 LAB7
LAB23 LAB55
LAB23 LAB57
LAB24 LAB2
LAB24 LAB3
LAB24 LAB39
LAB25 LAB14
LAB27 LAB5
LAB27 COM10
LAB34 LAB15
LAB35 LAB36
LAB35 LAB57
LAB39 LAB2
LAB42 COM8
LAB43 LAB6
LAB44 LAB56
LAB46 LAB47
LAB47 LAB56
LAB48 LAB6
LAB53 LAB12
LAB53 LAB24
LAB53 LAB54
LAB54 LAB55
LAB54 LAB57
LAB57 LAB18
LAB59 LAB12
LAB59 LAB24
LAB59 COM10
COM1 age
COM1 COM10
COM1 COM11
COM2 COM8
COM3 COM10
COM3 COM11
COM4 COM10
COM4 COM11
COM5 LAB19
COM5 COM1
COM5 COM10
COM5 PRO8
COM6 COM8
COM6 PRO3
COM7 LAB56
COM7 COM1
COM7 COM3
COM7 COM5
COM7 COM8
COM7 COM10
COM7 COM11
COM8 LAB24
COM8 COM1
COM8 COM11
COM8 PRO2
COM9 LAB56
COM9 LAB57
COM11 LAB38
COM11 COM5
COM11 COM10
COM11 PRO3
PRO1 LAB10
PRO1 PRO3
PRO1 PRO6
PRO2 COM4
PRO2

In [9]:
clients[0].score

{'acc': [0.819650067294751, 0.8048452220726783, 0.8115746971736204],
 'recall': [0.7614213197969543, 0.7715736040609137, 0.766497461928934],
 'auc': [0.8010403302281474, 0.7942117104553653, 0.7971681448838809],
 'f1': [0.6912442396313364, 0.6770601336302895, 0.6832579185520362],
 'score': [-63450.975181824055, -61499.939575924465, -61634.92720740024]}

In [ ]:
clients[0].score

In [ ]:
for i in fuse_dag_list[0].edges:
    print(i[0],"->",i[1],";")